### Ejercicio 18: Aceite

Una empresa elabora un aceite nuevo (REFI) refinando exclusivamente diferentes tipos de
aceite y mezclándolos. Los tipos de aceite se clasifican en dos categorías: vegetales (VEG1 y
VEG2) y no vegetales (OIL1, OIL2 y OIL3). Dependiendo del tipo de aceite, vegetal o no vegetal,
se requiere una línea de producción distinta para refinarlo, por lo que se puede refinar un máximo
de 200 toneladas de aceite vegetal y 250 de no vegetal. Además, se puede asumir que el coste del
refinamiento es nulo y que durante este proceso no se producen pérdidas de peso. Por otro lado,
existen restricciones de control de calidad que imponen cotas (inferior y superior) a la acidez
del producto final: 1.4 y 1.8 unidades, respectivamente. Se puede asumir que la acidez se mezcla
linealmente. La acidez y el coste de una tonelada de cada tipo de aceite se re
eja en la siguiente
tabla:
 Aceite | VEG1 | VEG2 | OIL1 | OIL2 | OIL3 
-----------------|---------------|---------------|---------------|---------------|----------------------------
 Coste  | 110           | 120           | 130           | 110           | 115         
 Acidez | 2,2           | 1,1           | 2             | 1,2           | 1,5 

Cada tonelada de producto final se vende a un precio de 150 euros. Plantea el modelo que
maximiza el beneficio neto.

$$
\begin{equation*}
    \begin{aligned}&\text{min}&& z=150\sum_v F_v+150\sum_o F_o-\sum_v coste_v F_v-\sum_o coste_o F_o \\
    & \text{sujeto a:} \\
    &  && \sum_v acidez_v F_v+\sum_o acidez_o F_o \leq 1.8 (\sum_v F_v+\sum_o F_o)  && \\
    &  && \sum_v acidez_v F_v+\sum_o acidez_o F_o \geq 1.4 (\sum_v F_v+\sum_o F_o)  && \\
    &  && F_v \in (VEG1, VEG2), F_o \in (OIL1, OIL2, OIL3) &&\\
    &  && F_v, F_o \geq 0 
    \end{aligned}
    \end{equation*}
$$

In [1]:
from pyomo.environ import *
import numpy as np
model = ConcreteModel()

In [2]:

model.veg = RangeSet(2)
model.oil = RangeSet(3)
Costeveg = {}
Costeoil = {}
Costeveg[1] = 110
Costeveg[2] = 120
Costeoil[1] = 130
Costeoil[2] = 110
Costeoil[3] = 115

Acidezveg = {}
Acidezoil = {}
Acidezveg[1] = 2.2
Acidezveg[2] = 1.1
Acidezoil[1] = 2
Acidezoil[2] = 1.2
Acidezoil[3] = 1.5

model.Costev = Param(model.veg, initialize = Costeveg)
model.Costeo = Param(model.oil, initialize = Costeoil)
model.Acidezv = Param(model.veg, initialize = Acidezveg)
model.Acidezo = Param(model.oil, initialize = Acidezoil)


model.pprint()

2 RangeSet Declarations
    oil : Dimen=1, Size=3, Bounds=(1, 3)
        Key  : Finite : Members
        None :   True :   [1:3]
    veg : Dimen=1, Size=2, Bounds=(1, 2)
        Key  : Finite : Members
        None :   True :   [1:2]

4 Param Declarations
    Acidezo : Size=3, Index=oil, Domain=Any, Default=None, Mutable=False
        Key : Value
          1 :     2
          2 :   1.2
          3 :   1.5
    Acidezv : Size=2, Index=veg, Domain=Any, Default=None, Mutable=False
        Key : Value
          1 :   2.2
          2 :   1.1
    Costeo : Size=3, Index=oil, Domain=Any, Default=None, Mutable=False
        Key : Value
          1 :   130
          2 :   110
          3 :   115
    Costev : Size=2, Index=veg, Domain=Any, Default=None, Mutable=False
        Key : Value
          1 :   110
          2 :   120

6 Declarations: veg oil Costev Costeo Acidezv Acidezo


In [3]:
model.Fv = Var(model.veg, domain = NonNegativeReals)
model.Fo = Var(model.oil, domain = NonNegativeReals)

In [4]:
model.benef = Objective(expr = 150*(sum(model.Fv[i] for i in model.veg)+sum(model.Fo[j] for j in model.oil))-sum(model.Fv[i]*model.Costev[i] for i in model.veg)-sum(model.Fo[j]*model.Costeo[j] for j in model.oil), sense = maximize)

In [5]:
model.maxprodv = Constraint(expr = sum(model.Fv[i] for i in model.veg) <= 200)
model.maxprodo = Constraint(expr = sum(model.Fo[i] for i in model.oil) <= 250)
model.acup = Constraint(expr= sum(model.Fv[i]*model.Acidezv[i] for i in model.veg)+sum(model.Fo[i]*model.Acidezo[i] for i in model.oil) <=1.8*(sum(model.Fv[i] for i in model.veg)+sum(model.Fo[j] for j in model.oil)) )
model.aclo = Constraint(expr= sum(model.Fv[i]*model.Acidezv[i] for i in model.veg)+sum(model.Fo[i]*model.Acidezo[i] for i in model.oil) >=1.4*(sum(model.Fv[i] for i in model.veg)+sum(model.Fo[j] for j in model.oil)) )



In [6]:
results = SolverFactory('glpk').solve(model)
model.pprint()
results.write()   

2 RangeSet Declarations
    oil : Dimen=1, Size=3, Bounds=(1, 3)
        Key  : Finite : Members
        None :   True :   [1:3]
    veg : Dimen=1, Size=2, Bounds=(1, 2)
        Key  : Finite : Members
        None :   True :   [1:2]

4 Param Declarations
    Acidezo : Size=3, Index=oil, Domain=Any, Default=None, Mutable=False
        Key : Value
          1 :     2
          2 :   1.2
          3 :   1.5
    Acidezv : Size=2, Index=veg, Domain=Any, Default=None, Mutable=False
        Key : Value
          1 :   2.2
          2 :   1.1
    Costeo : Size=3, Index=oil, Domain=Any, Default=None, Mutable=False
        Key : Value
          1 :   130
          2 :   110
          3 :   115
    Costev : Size=2, Index=veg, Domain=Any, Default=None, Mutable=False
        Key : Value
          1 :   110
          2 :   120

2 Var Declarations
    Fo : Size=3, Index=oil
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   0.0 :  None : False : False : NonNegativeR